# import

In [240]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
assert gensim.models.word2vec.FAST_VERSION > -1
from gensim.models import Word2Vec
import datetime
import requests
import re
from bs4 import BeautifulSoup

# Data 준비하기

In [244]:
df_contest = pd.read_csv('contest_process.csv') # 대외활동 데이터
id_to_name = pd.Series(df_contest.company_name.values, index = df_contest.idx.values)
name_to_id = pd.Series(df_contest.idx.values, index = df_contest.company_name).to_dict()


df_contest.head(5)

,Unnamed: 0,idx,contest_name,company_name,company_type,공공기관/공기업,기업형태_기타,대기업,비영리단체/협회/재단,중견기업,...,benefit,기타,상장 수여,실제상용화,인턴/정규직채용,입사시 가산점,전시기회,해외연수,시상규모(만원),시상 규모
0,0,80219,제11회 백년서평 공모,백년어서원,기업형태_기타,0,1,0,0,0,...,기타,1,0,0,0,0,0,0,95,소
1,1,80272,[문체부] 2022 문화 체육 관광 분야 사회적경제기업 우수사례 발굴 공모전 (~3...,(주)필더필컴퍼니,기업형태_기타,0,1,0,0,0,...,"기타, 상장 수여",1,1,0,0,0,0,0,1100,대
2,2,80296,제 4회 멸종위기야생생물 상상그림 및 상상뉴스 공모전(~3/11),스펙토리,공공기관/공기업,1,0,0,0,0,...,"기타, 상장 수여",1,1,0,0,0,0,0,340,중
3,3,80314,LG유플러스 아이들나라 창작 그림책 공모전,"LG유플러스, 교보문고, 한국그림책출판협회, 보림출판사",대기업,0,0,1,0,0,...,기타,1,0,0,0,0,0,0,10000,대
4,4,80341,[인턴연계] 2022년 인천e음 배달서비스 홍보 영상 공모전,코나아이(주),중견기업,0,0,0,0,1,...,인턴/정규직채용,0,0,0,1,0,0,0,450,중


# 필요없는 열제거

In [245]:
df_contest=df_contest.drop(["시상규모(만원)"],axis='columns')
df_contest=df_contest.drop(["Unnamed: 0"],axis='columns')
df_contest=df_contest.drop(["company_type"],axis='columns')
df_contest=df_contest.drop(["benefit"],axis='columns')
df_contest=df_contest.drop(["field"],axis='columns')
df_contest=df_contest.drop(["target"],axis='columns')
df_contest.head(3)

,idx,contest_name,company_name,공공기관/공기업,기업형태_기타,대기업,비영리단체/협회/재단,중견기업,중소기업,UCC,...,직장인/일반인,청소년,기타,상장 수여,실제상용화,인턴/정규직채용,입사시 가산점,전시기회,해외연수,시상 규모
0,80219,제11회 백년서평 공모,백년어서원,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,소
1,80272,[문체부] 2022 문화 체육 관광 분야 사회적경제기업 우수사례 발굴 공모전 (~3...,(주)필더필컴퍼니,0,1,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,대
2,80296,제 4회 멸종위기야생생물 상상그림 및 상상뉴스 공모전(~3/11),스펙토리,1,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,중


# train set, test set 분리

In [246]:
df_contest_train, df_contest_test = train_test_split(df_contest, random_state = 15688, test_size=0.30)



In [247]:
df_contest_train.head(3)

,idx,contest_name,company_name,공공기관/공기업,기업형태_기타,대기업,비영리단체/협회/재단,중견기업,중소기업,UCC,...,직장인/일반인,청소년,기타,상장 수여,실제상용화,인턴/정규직채용,입사시 가산점,전시기회,해외연수,시상 규모
1122,91354,웨스턴디지털 Show Your WD 영상 공모전,주식회사 메이캔,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,소
1004,90403,2022 안산의 책 독서감상작 전국 공모전,안산 중앙도서관,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,소
1146,91568,제2회 뮤튜브 영상 공모전,숭실대학교 한국기독교박물관,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,중


# 같은 대외활동끼리 묶기

In [251]:
# 비교할 때 제외할 열들의 이름 리스트
def data_splitter(df):
    exclude_cols = ['idx','contest_name', 'company_name']

    grouped_cols = [col for col in df_contest.columns if col not in exclude_cols]  # 제외할 열을 제외한 열들의 리스트

    df_grouped = df_contest.groupby(grouped_cols).apply(lambda x: x.idx.tolist())
    
    # 결과값을 str 타입으로 저장한 이중 배열
    array_of_groups = df_grouped.apply(lambda x: [str(val) for val in x]).tolist()
    return array_of_groups

In [252]:
pd.options.mode.chained_assignment = None
contest_ls = data_splitter(df_contest_train)

# 데이터 랜덤으로 섞고, 유효한 데이터 골라내기

In [253]:
for contest in contest_ls:
    random.shuffle(contest)

In [254]:
filtered_array = [group for group in contest_ls if len(group) > 1]

In [255]:
len(filtered_array)

595

# Model 생성 및 학습

In [273]:
model = Word2Vec(
    sentences = filtered_array, # 전처리된 리스트를 파라미터로 사용
    epochs = 70, # epoch
    min_count = 1, # 
    vector_size = 100, # hidden layer 사이즈
    sg = 1, # skip-gram을 사용하면 1
    hs = 0, # negative sampling을 사용하면 0
    negative = 5, # 0 이상일 경우 negative sampling을 사용함.
    window=5,  # 주변 단어 창 크기
    workers=4  # 사용할 CPU 스레드 수
)

model.save('item2vec_contest')

In [274]:
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
model = Word2Vec.load('item2vec_contest')
word_vectors = model.wv

# 추천

In [260]:
def recommender(positive_list=None, negative_list=None, topn=20):
    recommend_contest_ls = []
    
    for contestId, prob in model.wv.most_similar_cosmul(positive=positive_list, negative=negative_list, topn=topn):
        recommend_contest_ls.append(contestId)
    return recommend_contest_ls

In [261]:
reco = [80826]
reco_str = [str(num) for num in reco]
recommandations = recommender(positive_list=reco_str, topn=5)
reco_int = [int(value) for value in recommandations] # str -> int
print(reco_int)

[105084, 97930, 100310, 85869, 109079]


In [262]:
# 입력 데이터
filtered_activity_names = df_activity.loc[df_activity['idx'].isin(reco), 'company_name'].tolist()
print(filtered_activity_names)

['KTV 한국정책방송원']


In [263]:
# 출력 데이터
filtered_activity_names = df_activity.loc[df_activity['idx'].isin(reco_int), 'company_name'].tolist()
print(filtered_activity_names)

['정선군 / 오마이필름', '국립공원공단', '구미시청', '통일부 / 모두를위한극장 공정영화협동조합', '울산광역시 북구']


In [265]:
print(model.wv.most_similar(["80826"]))

[('105084', 0.9967342615127563), ('97930', 0.9966229796409607), ('100310', 0.9965788722038269), ('85869', 0.9961397051811218), ('109079', 0.9959255456924438), ('101189', 0.9955222606658936), ('86666', 0.9953718185424805), ('99870', 0.994710385799408), ('104191', 0.9941415786743164), ('104435', 0.9931982755661011)]
